In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

from langchain_core.output_parsers import StrOutputParser

In [1]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


In [15]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens = 500)

In [ ]:
# Carregar PDF

pdf_link = "data/os-sertoes.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [17]:
len(pages)

14

In [18]:
# Splitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200, length_function=len, add_start_index=True)



In [19]:
# Storage

store = InMemoryStore()
vectorstore = Chroma(embedding_function=embeddings, persist_directory='childVectorDB')

In [20]:
parent_document_retriever = ParentDocumentRetriever(
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    vectorstore=vectorstore,
    docstore=store
)
parent_document_retriever.add_documents(pages, ids=None)

In [21]:
parent_document_retriever.vectorstore.get()

{'ids': ['7ee8fd8f-6f95-421f-b1e4-8c67619c27db',
  'ae3f29d3-dc21-4c38-b020-ad5edb8a1b01',
  '4f442f27-8d64-4042-8666-619eb153c847',
  '2d8b1b99-5ef5-411d-9f51-76a51996eb6d',
  '7a954fad-5e21-4482-9792-a88a0d056465',
  '619be9e0-b2c6-4a83-b312-4737a4b0dfc1',
  'a062b52a-e41c-498a-bd76-b36f6f3044b5',
  'b187d020-d7ab-4d94-aa60-bbb6c42a714d',
  'fd87f1ef-e4b2-4e0d-acc8-fb3544dcc0c3',
  '19bfea67-de38-453b-893b-63ba391fd754',
  '7527ebb5-7189-4cbe-b304-0e462b1376a4',
  '44e7a9d9-8646-453e-9df7-07f40d06e652',
  '303d1f12-94a5-4852-89b6-20d5aa49c007',
  'fe266565-1338-4a4a-8d3f-5e5b3d106e7b',
  '5f6d5737-11de-4958-b44f-3d03bdd4cfa8',
  '3ecd0a71-1f9c-4005-9989-48d9afb6b0ac',
  'c61d361e-ed82-401e-b2e6-4b63d2361e68',
  '03e8b2d7-a65f-4cdc-a2a0-f39eddbe58f0',
  '55614948-936b-4d25-974b-1c166f71df4e',
  'c353bef0-ebf0-45c5-8b52-ab157db26e90',
  '43f97db7-58cd-4d8f-9aa4-8c9076e8a8c6',
  'fed6f7db-17a8-4d44-871d-7a05b576de58',
  '7582a252-bb6f-4d49-b5f1-3af7d4b12022',
  '90d5537f-f907-404b-bed6-

In [24]:
TEMPLATE = """
Você é um assistente de IA que ajuda a responder perguntas sobre o conteúdo de documentos legais. Você deve usar o contexto fornecido para responder às perguntas.
Query: {question}
Context: {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [26]:
setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": parent_document_retriever})

output_parser = StrOutputParser()

In [27]:
parent_chain_retriever = setup_retrieval | rag_prompt | llm | output_parser

In [28]:
parent_chain_retriever.invoke("Quais são os principais desafios éticos associados à IA?")

'Os principais desafios éticos associados à IA incluem a falta de parâmetros mais robustos de direitos e deveres para a concretização dos princípios e objetivos enunciados nos projetos legais, a necessidade de expandir as definições para lidar com os enormes potenciais econômicos e sociais e os riscos para os direitos e liberdades fundamentais associados à inteligência artificial, a questão da responsabilização em caso de erros cometidos por sistemas de IA e a necessidade de supervisão e revisão humana ao longo de todo o ciclo de vida do sistema para garantir a transparência e finalidade das operações.'